In [2]:
import bs4
from dotenv import load_dotenv
from langchain import hub
from operator import itemgetter
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from utils import format_qa_pair, format_qa_pairs

from colorama import Fore
import warnings

warnings.filterwarnings("ignore")

load_dotenv()

# LLM
llm = ChatOpenAI()

In [3]:
# load documents (data source: https://blog.langchain.dev/)
loader = WebBaseLoader("https://blog.langchain.dev/deconstructing-rag/")
docs = loader.load()

loader = WebBaseLoader("https://blog.langchain.dev/reflection-agents/")
docs.extend(loader.load())

#  split documents, create vector store and load embeddings
loader = WebBaseLoader(
    web_paths=("https://blog.langchain.dev/reflection-agents/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("article-header section", "article-header__content", "article-header__footer")
        )
    ),
)
blog_docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)
splits = text_splitter.split_documents(blog_docs)

# Index and load embeddings
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

# Create the vector store
retriever = vectorstore.as_retriever()

### Decomposition

In [8]:
template = """You are a helpful assistant trained to generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answered in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)


def generate_sub_questions(query):
    """ generate sub questions based on user query"""
    pass 
    # Chain
    generate_queries_decomposition = (
        prompt_decomposition 
        | llm 
        | StrOutputParser()
        | (lambda x: x.split("\n"))
    ) 

    # Run
    sub_questions = generate_queries_decomposition.invoke({"question": query})
    questions_str = "\n".join(sub_questions)
    print(Fore.MAGENTA + "=====  SUBQUESTIONS: =====" + Fore.RESET)
    print(Fore.WHITE + questions_str + Fore.RESET + "\n")
    return sub_questions 

In [10]:
sub_questions = generate_sub_questions("How is context improving AI systems")

=====  SUBQUESTIONS: =====
1. What role does context play in enhancing the performance of AI systems?
2. How can context be integrated into AI algorithms to improve accuracy and efficiency?
3. What are some real-world examples of AI systems benefiting from contextual information?



### Answer Subquestions Recursively

In [11]:
template = """Here is the question you need to answer:

\n --- \n {sub_question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {sub_question}
"""
prompt_qa = ChatPromptTemplate.from_template(template)


def generate_qa_pairs(sub_questions):
    """ ask the LLM to generate a pair of question and answer based on the original user query """
    q_a_pairs = ""

    for sub_question in sub_questions:
        # chain
        generate_qa = (
            {"context": itemgetter("sub_question") | retriever, "sub_question": itemgetter("sub_question"), "q_a_pairs": itemgetter("q_a_pairs")}
            | prompt_qa 
            | llm 
            | StrOutputParser()
        )
        answer = generate_qa.invoke({"sub_question": sub_question, "q_a_pairs": q_a_pairs})
        q_a_pair = format_qa_pair(sub_question, answer)
        q_a_pairs = q_a_pairs + "\n --- \n" + q_a_pair 

In [12]:
generate_qa_pairs(sub_questions)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


=====  QUESTION/ANSWER PAIRS: =====
Question: 1. What role does context play in enhancing the performance of AI systems?
Answer: Context plays a crucial role in enhancing the performance of AI systems by providing relevant information and parameters for the system to make informed decisions. In the case of reflection agents, context can help in guiding the reflection techniques used to improve the quality and success rate of agents. By understanding the context in which the AI system is operating, developers can tailor the algorithms and strategies to optimize performance and achieve better outcomes. This highlights the importance of considering context in the design and implementation of AI systems to ensure they are effective and efficient in various scenarios.

 


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


=====  QUESTION/ANSWER PAIRS: =====
Question: 2. How can context be integrated into AI algorithms to improve accuracy and efficiency?
Answer: Context can be integrated into AI algorithms to improve accuracy and efficiency by providing relevant information and parameters for the system to make informed decisions. In the case of reflection agents, context plays a crucial role in guiding the reflection techniques used to enhance the quality and success rate of agents. By understanding the context in which the AI system is operating, developers can tailor the algorithms and strategies to optimize performance and achieve better outcomes. This highlights the importance of considering context in the design and implementation of AI systems to ensure they are effective and efficient in various scenarios. In essence, by incorporating context into AI algorithms, developers can better adapt the system to different situations, leading to improved accuracy and efficiency.

 


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


=====  QUESTION/ANSWER PAIRS: =====
Question: 3. What are some real-world examples of AI systems benefiting from contextual information?
Answer: Some real-world examples of AI systems benefiting from contextual information include:

1. Personalized recommendations: AI systems used by streaming platforms like Netflix or music apps like Spotify benefit from contextual information such as user preferences, viewing history, time of day, and location to provide personalized recommendations for content or songs.

2. Autonomous vehicles: AI systems in autonomous vehicles utilize contextual information from sensors, cameras, GPS, and traffic data to make real-time decisions while driving. This context helps the vehicles navigate safely, avoid obstacles, and optimize routes.

3. Chatbots and virtual assistants: AI-powered chatbots and virtual assistants like Siri, Alexa, or Google Assistant rely on contextual information such as user input, previous interactions, and current tasks to provide re

### Answer Individually

In [15]:
# RAG prompt = https://smith.langchain.com/hub/rlm/rag-prompt
prompt_rag = hub.pull("rlm/rag-prompt")


def retrieve_and_rag(prompt_rag, sub_questions):
    """RAG on each sub-question"""
    rag_results = []
    for sub_question in sub_questions:
        retrieved_docs = retriever.get_relevant_documents(sub_question)

        answer_chain = (
            prompt_rag
            | llm
            | StrOutputParser()
        )
        answer = answer_chain.invoke({"question": sub_question, "context": retrieved_docs})
        rag_results.append(answer)
    
    return rag_results, sub_questions

/data/rag/lib/python3.10/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


### Summarize and Answer

In [21]:
# Prompt
template = """Here is a set of Q+A pairs:

{context}

to use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)


# Query
def query(query):
    # generate optimized answer for a given query using the improved subqueries
    question = query
    queries = [
        "How is context improving AI systems",
        "What are the two main components involved in Basic Reflection",
        "Explain the steps involved in the Reflexion loop"
    ]
    sub_questions = generate_sub_questions(query)

    #generate_qa_pairs(sub_questions)
    #answers, questions = retrieve_and_rag(prompt_rag, sub_questions)
    #context = format_qa_pairs(questions, answers)
    
    context = generate_qa_pairs(sub_questions)

    final_rag_chain = (
        prompt
        | llm
        | StrOutputParser()
    )

    return final_rag_chain.invoke({"question": question, "context": context})

In [22]:
query("How is context improving AI systems")

=====  SUBQUESTIONS: =====
1. What role does contextual information play in enhancing the performance of AI systems?
2. How do AI systems utilize contextual cues to make more accurate predictions or decisions?
3. What are some examples of successful implementations of context-aware AI systems in various industries?



Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


=====  QUESTION/ANSWER PAIRS: =====
Question: 1. What role does contextual information play in enhancing the performance of AI systems?
Answer: Contextual information plays a crucial role in enhancing the performance of AI systems by providing relevant data and insights that allow the system to make more informed decisions. For example, in the case of reflection agents as mentioned in the provided document, contextual information can be used to improve the quality and success rate of agents by guiding their actions based on past experiences and outcomes. By leveraging contextual information, AI systems can adapt and learn from their environment, leading to more efficient and effective decision-making processes. Ultimately, contextual information helps AI systems to better understand and navigate complex situations, leading to improved overall performance.

 


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


=====  QUESTION/ANSWER PAIRS: =====
Question: 2. How do AI systems utilize contextual cues to make more accurate predictions or decisions?
Answer: AI systems utilize contextual cues to make more accurate predictions or decisions by leveraging relevant data and insights from their environment. For example, in the case of reflection agents, contextual information is used to guide the actions of the agents based on past experiences and outcomes. By analyzing and understanding the context in which they are operating, AI systems can adapt and learn, leading to more informed decision-making processes. This ability to utilize contextual cues allows AI systems to better understand complex situations and make more accurate predictions or decisions, ultimately improving their overall performance.

 


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


=====  QUESTION/ANSWER PAIRS: =====
Question: 3. What are some examples of successful implementations of context-aware AI systems in various industries?
Answer: Some examples of successful implementations of context-aware AI systems in various industries include:

1. Healthcare: AI systems that leverage contextual information such as patient history, symptoms, and environmental factors to provide personalized treatment recommendations and predict potential health issues.

2. Finance: AI systems that analyze market trends, customer behavior, and economic indicators to make real-time investment decisions, detect fraud, and provide personalized financial advice.

3. Retail: AI systems that use contextual information like customer preferences, purchase history, and current trends to personalize marketing campaigns, optimize inventory management, and improve the overall shopping experience.

4. Manufacturing: AI systems that utilize contextual data from sensors, production lines, and supply

'Context is improving AI systems by providing a deeper understanding of the environment in which the AI operates. By incorporating contextual information into the algorithms, AI systems are able to make more informed decisions and predictions. This allows them to deliver more accurate and relevant results, ultimately improving their overall performance and effectiveness.'